## Import required packages

In [1]:
import os
import datetime

import tensorflow as tf

from data.abc import ABCPreProcessor, ABCTokenizer
from models.symbolic.rnn import FolkLSTM

## Required files and directories

In [2]:
BASE_DIR = "/home/richhiey/Desktop/workspace/projects/AI_Music_Challenge_2020/"
SRC_DIR = os.path.join(BASE_DIR, 'AI-Music-Generation-Challenge-2020')
ABC_DATA_DIR = os.path.join(BASE_DIR, "datasets", "abc_data")
AUDIO_DATA_DIR = os.path.join(BASE_DIR, "datasets", "audio")
ABC_TFRECORD_DIR = os.path.join(BASE_DIR, "tfrecords", 'abc')
AUDIO_TFRECORD_DIR = os.path.join(BASE_DIR, "tfrecords", 'audio')
PROCESSED_ABC_FILENAME = 'processed-abc-files'
FOLK_LSTM_DIR = os.path.join(SRC_DIR, 'configs', 'lstm')
TENSORBOARD_DIR = os.path.join(FOLK_LSTM_DIR, 'tensorboard')

## Load preprocessed dataset

In [3]:
preprocessor = ABCPreProcessor(ABC_TFRECORD_DIR, PROCESSED_ABC_FILENAME)
preprocessed_dataset = preprocessor.load_tfrecord_dataset()
print(preprocessed_dataset)

<MapDataset shapes: ({tune_length: ()}, {input: (None, None), output: (None, None)}), types: ({tune_length: tf.int64}, {input: tf.int64, output: tf.int64})>
<MapDataset shapes: ({tune_length: ()}, {input: (None, None), output: (None, None)}), types: ({tune_length: tf.int64}, {input: tf.int64, output: tf.int64})>


# Folk-LSTM

In [ ]:
FOLK_LSTM_DIR = os.path.join(SRC_DIR, 'configs', 'lstm')

dataset = preprocessor.prepare_dataset(preprocessed_dataset)
data_dims = preprocessor.get_data_dimensions(ABC_TFRECORD_DIR)
model = FolkLSTM(FOLK_LSTM_DIR, data_dims, ABC_TFRECORD_DIR)

print(data_dims)
print(model.get_configs())

model.train(dataset)

/home/richhiey/Desktop/workspace/projects/AI_Music_Challenge_2020/AI-Music-Generation-Challenge-2020/configs/lstm/lstm.json
{'tune_embedding_size': '16', 'rhythm_embedding_size': '16', 'key_embedding_size': '16', 'meter_embedding_size': '16', 'bidirectional': 'False', 'rnn': DictWrapper({'num_layers': '1', 'num_units': '512', 'unit_type': 'lstm'})}
{'max_timesteps': 511, 'tune_vocab_size': 168}
{'tune_embedding_size': '16', 'rhythm_embedding_size': '16', 'key_embedding_size': '16', 'meter_embedding_size': '16', 'bidirectional': 'False', 'rnn': DictWrapper({'num_layers': '1', 'num_units': '512', 'unit_type': 'lstm'})}
Initializing from scratch.
tf.Tensor(5.124014, shape=(), dtype=float32)
---------- Generated Output -----------
7=e'=e'K:AK:Ddor^g'_c'^g'_GB'}/2<5555=C=C=C_c'_c'C,C,_bC,f'_c'_c'f'_c'_G_bf'^g'_c'_G_GB'_G/2<5555=C=C=C||=C=C||/2<^g||=bB'B'B'(_d'_d'||||=C=C=C|||=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C=C|=C=C=C=C|||B'=C=C=a=a(^G,_G^G,=a^G,^g_B[_B=a_e_E,_E,B'B'B'
.................

tf.Tensor(4.166358, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:GAddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddd
.......................................
tf.Tensor(4.154065, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:GAddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddd
.......................................
tf.Tensor(4.1472664, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:GA|dddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddd
.......................................
tf.Tensor(4.0536404, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:GAd||dd||dddddddddd|d||ddd|d||||d|d||dddddddddd|d||d|ddd||||ddddddddddddddddddd|dddddddddd|dddddddddd|dddddd|ddddddddddddd|d|dddddddddddd|dddddddd

tf.Tensor(3.672791, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:GA|AB|G|GG|dAAAAGAGd|A|GGGGGAdAAB|dAGAGGAd|G|GG|dAAAAGAB|e|ddBAAGABGA|A|GG|dedfffddddfgffgfdgggdBBdgdgddgd||ffddddfgffdAdBAAGABAA|A|GG||d|gfgfgdffdddAdBAAGABGA|A|GG|
.......................................
tf.Tensor(3.5815835, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:GA|A|GGGGGAAGAAGGA|GFGFGAGABAAGA|GGGGGAAGGAABBedBBAGAGGGGAdBdB|ffgfeggggdddBABddgdggfgdddBfffdBdAAGAABBedBBAGAGGGGAd|A|GGAGGABBBAABAAGAGGGAGGABAAGAAAGGA|GGAGAAAAABffdgdBBAGAGGGGAdBd|B|fgfdgdgdgfg|gddddddgf|ddBA|GGABAAAAAABBBBdgfdgddBAGAGGGGA
.......................................
tf.Tensor(3.6430888, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:GA|AAGG|A|AAd|G|DA|AAGG|A|AAAAdBBBAGAB||AGG|A|AAd|GAdddfggfdBAGA|GAAdBBBAGA||||AABdd|AAABBddddggggddgdBAGABddddegggddggg|gggddBddddggggddgdBBddgddBAGA|AABdBAAABBddddegggddgdBAGABddddegggddggg|gggddBddggddg|ddddBd|dddBdddddgggddAdBAGAG

tf.Tensor(3.2208335, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:G(|G|GAGAdegd|G:|AdAcd|AAd|AA|G|B|AAGAdegd|G:|AdAcd|AAdAGAG:|</s>d|B||AAABAddddde||edge|dddddeeegd|Ad|g||gdeeeg|dAdAGAG:|</s>
.......................................
tf.Tensor(3.1608768, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:G(|A|GAeA:||e|e|AA|:|dedcAAA|B|AAGAe|:||d|eBdg|g|ed|AAA|</s>ddg|gg|egg|gd|edg|g|geggg|d|edg|gg|egg||gde|dg|g|ed|AAA|</s>ddAd|dAdAB|eAdd|AA|BdAABdAd|dAdAB|eAdd|AAAABAGBdAd|dAdAB|eAdd|AA|BdAABBdg|g|ed|AAA|</s>
.......................................
tf.Tensor(3.2186043, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:G(|GBFA|GG:||e|AB|ABedfd|ABe|AAFAAB|BAGG:||e|ABcABedfd|ABe|BAGG:|</s>d|Gd|dd|e|cd|BAGBABAABAd|GAAGFGB|A|ed|d|GAAdde|ABfdAd|B|GG:|</s>
.......................................
tf.Tensor(3.2164142, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:G(|AFGFGFBFGFFGFGFAFFG:|AB|BBAAFGFAFGGGF

tf.Tensor(2.8296688, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:GAABABAA|AA|dd|A|2A|BAAA2A|BAGD|:||AdAA|Addd|AA2A|BAAA|D|BFAA:|</s>d|dgefe|f|gde|A|2A|dfgggg|ffdddd|dg|fe|f|gde|A|2A|d|AA2D|BFAA:|</s>
.......................................
tf.Tensor(2.852877, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:DA|BBGA|BFDA|:||FFBBGA|GBFD|2D|DFBAAd|BFDA|:||FFAAdBdAAABGG|2D|</s>|BAdddded||d|BFBA|:||fededA|BFBAB:||fefggd|BdFG|2A|BAGD2D|</s>
.......................................
tf.Tensor(2.8320885, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:DDAGAAAAdFGAdAAddeAA2A|BAGA|GAdABGdA|BdGAAGAdFGAdAdddeAA|A|BGAdBd|fddgfffBAddedAAGA</s>d|BAAAAdFGAdA:|ddeAd|A|BfBAAA|dBAAdA|BfBAAAAdFGAdA:|ddeAd|A|BGAd2d|fddgfffBAddedAAGA</s>
.......................................
tf.Tensor(2.7721214, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:D(|dedd|G|GGGA2:||fedd|G|GGGA2:||fedd|G|BGBdgg|fedd|G|BGGA2:|</s>(|d

...

tf.Tensor(1.0445528, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:DA|dFDDFD|BeBeFA|DFFG2A|DFFG2A|DFDDFA|AcAeFA|DFFD2:||AFADF:|d|BFFD2B|cccB2:||AFFG2A|DFFG2A|DFFDD2:||cccB2:||cFAD2:||cFAD2:||:2gfBB2B|AAAAGd|eFAD2:||cFAD2:|</s>
.......................................
tf.Tensor(1.040716, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:D(AF)F)A||||deAAFF|AFdGFA|DAAGFD|AAFEFG)|F||dcAAFA|AFAdefB|deFF2D||:2GAdefg|fddd2:|dd|fedddd|ecAAAg|fddd2e||ga}gfeafg|ff<a)fgfg|fgddAA|BBGdefg|fddd2:|</s>2gcdefB|deFFGD|</s>
.......................................
tf.Tensor(1.0643282, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:D(||deBeAE|AAAdFA|dFGGEA|BFdAFc|deAeAE|AAAdFA|dFGGGA|BGEG2:||:d|efgdBB|edcBAd|ecAAeG|AAAAGD|DfgdBB|cccBAd|ecee2g|fddd2:|</s>
.......................................
tf.Tensor(1.057555, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:DD|GAEGBc|dAGdBd|gefeeed)BAdcc)AAA2:||GAGGBAA|B)AAG

---------- Generated Output -----------
M:6/8K:G(|dcdAA2BA|DAADD2D|DAADDBAD|GAGc2A|dBcBA2BA|DAADD2:||DAADD2A||GEEDD2:||:|BedBefA|GedBedA|GABcBdA|BBBGcAE|GGBcddd|cc}g)dcd2d|GBBDD2BD|GEADD2:|</s>
.......................................
tf.Tensor(0.89723, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:G(|BAGAGD|DEGDED|AAAcBA|dBGGGG|BAGAGD|DEGD2D|AGBAGD|DAG2:||:d|AAABG2d|ededBG|AAAcGc|eGBA2D|DAAGG2d|ededBG|AABA2D|DAG2:|</s>
.......................................
tf.Tensor(0.9308505, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:G(|DFADD2AD|DFBAAABF|FFAAFB|FAAGED|DEEDD2AD|DFBAAABF|FFFGFD|DADD2:||:(|deef2gdccedaff|eAeeec|Bfcddc|dcdddd|cAecAF|AdGG2D|DADD2:|</s>
.......................................
tf.Tensor(0.88144785, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:G(||AFE|DFAAFE|DcBAFF|DAGF2D|FcAAFF|DFAAFF|DcBAFc|dcAAFD||:F2EAFD|DFFGFD|DEEAFD|DFFG2D|BEEAFD|DFFGFD|DEEAFc|dcBAED|</s>(EAdFD|FFAdED|FFAdFD|FFFG2D|FFAd

tf.Tensor(0.8360253, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:D(||dcGGBG|AAeeBG|AAeBdB|AAAFGD|DcGGAA|dfggdd|cddcAF|GGGG2:||:|FBggfg|fedfd|dFedgf|eccB2d|(EB}gfeafffedfd|ddeeeA|(dGG2:|</s>
.......................................
tf.Tensor(0.80431616, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:D(|cdeedB|deGAAE|GAGGcB|AABA2d|edeedB|deGAAE|GAGAAB|AGGG2:||:(|dBdgc2|fgae{a}g))||dfeddB|deGAAE|GAdgcf|fgae{a}g))||dBGAAE|GEGG2:|</s>
.......................................
tf.Tensor(0.84736156, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:D(|cdeedB|deGA2B|ABGG2d|eGGA2:||fedecg|agfecd|geeeBG|edAA2:|B|agabfg|agaecd|gfeeBf|gfeaBe|fffabfg|agaecd|gfeaBG|eaAA2:|</s>2gbaefe|dedcdB|(feaBG|eaAA2:|</s>
.......................................
Saved checkpoint for step 323: /home/richhiey/Desktop/workspace/projects/AI_Music_Challenge_2020/AI-Music-Generation-Challenge-2020/configs/lstm/ckpt/ckpt-28
tf.Tensor(0.7363937, shap

tf.Tensor(0.6653741, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:G(|DFAAce|fefB2F|AFAA2A|FFAE2:||DFAd2e|fefB2F|AFAA2G|AEDD2:||:|cfaafd|dfgfef|eeaagd|gAee2f|geeagd|gfgfef|eAecGA|BEDD2:|</s>
.......................................
tf.Tensor(0.69350606, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:D(|GGABBBAB|BGBBcBA|dBGGGA|BGGABD|DGGBcBAB|BGBBcBA|dBGGGA|BGGG2:||:d|GBBaefg|aedcAG|GBAgeeg|agddBG|GGABBBAB|BEBAcBe|dBGGGA|BGGG2:|</s>
.......................................
tf.Tensor(0.67005444, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:DA|AAAf|gdcB2G|AEAA2F|AEDD2A|DEAd2f|gdcB2F|AEAd2f|fdfd2:||:fddfee)ff|ggf).gffe)ff|fedcea)ff|ggf).f(fg|fddcea)ff|ggf).gffe)ff|fedc2f|fefd2:|</s>
.......................................
tf.Tensor(0.6469456, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:DAAG)|AFDDFA|dGdecA|BddeFF|AAGAFA|AFADFA|dGdecA|BddeEA|D2DD2:||:(|fefdfG|AABd2e|fdddef|gfeafd|bede2d|AF

tf.Tensor(0.59044623, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:GD||cdg|dBGBAG|ABGBdg|dBGGABGBA}G)|GBGBfA|BBGB{d}B)AG|ABBc2e|dBGG{d}A))G||:gfBgdd|dfBBcB|FfBgdd|fBGG{BGBA}G)|AfBgdc|ffBBcB|FBBc2e|dBGG{B}A))G|</s>
.......................................
tf.Tensor(0.62437963, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:GEAB)|EFAAAc|BcAAAA|BAAAAA|BBGB2A|BBAAA|BcAA2A|BEEA2d|eBAA2:||:eed)|a2efe|dBeedA|B>AA2A|BBGB2d|d^ggfe|dBAecA|B>AA2d|eBAA2:|</s>eed)e2)|{2)|)(BdAA))BBB})G||A2ABA|BBAB2AA|)e2)({2c))(B||A))BBB}))||A2A2d|eBAA2:|</s>
.......................................
tf.Tensor(0.6014896, shape=(), dtype=float32)
---------- Generated Output -----------
M:6/8K:Gcgd)|cAAEBE|GAEGED|A2EGBE|GGddBf|cAAG2E|GAEG2D|A2ed2d|BAAA2:|d|gfgged|edAAde|ffag2e|ddcg2e|abfgged|edAAde|fedced|ccAA2:|</s>2gfgdff|gfedBB|cAeeff|eAAA2:|</s>
.......................................
tf.Tensor(0.57860684, shape=(), dtype=float32)
---------- Generated Output ------

# Generate 10000 double jigs!

In [ ]:
model.generate_10000_double_jigs()